In [1]:
import fastText
import math
import linecache
import numpy as np 
from random import shuffle
from numpy import random
from keras.models import Sequential, Model
from keras.callbacks import ModelCheckpoint
from keras.layers import *
from keras import regularizers
from keras.utils import to_categorical
from keras.preprocessing.sequence import pad_sequences
from keras.initializers import RandomUniform
import re
from sklearn.metrics import f1_score, recall_score, precision_score, accuracy_score


/home/jindal/miniconda3/envs/NER2/lib/python3.5/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=1

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=1


In [3]:
ft = fastText.load_model("/home/jindal/notebooks/fastText/wiki.de.bin")

nb_embedding_dims = ft.get_dimension()
nb_sequence_length = 75

In [ ]:
SINGLE_ATTENTION_VECTOR = True
def attention_3d_block(inputs):
    # inputs.shape = (batch_size, time_steps, input_dim)
    input_dim = int(inputs.shape[2])
    a = Permute((2, 1))(inputs)
    a = Reshape((input_dim, nb_sequence_length))(a) # this line is not useful. It's just to know which dimension is what.
    a = Dense(nb_sequence_length, activation='softmax')(a)
    if SINGLE_ATTENTION_VECTOR:
        a = Lambda(lambda x: K.mean(x, axis=1), name='dim_reduction')(a)
        a = RepeatVector(input_dim)(a)
    a_probs = Permute((2, 1), name='attention_vec')(a)
    output_attention_mul = multiply([inputs, a_probs], name='attention_mul')
    return output_attention_mul

In [4]:
def twitter_tokenizer(textline):
    textline = re.sub('@[\w_]+', 'USER_MENTION', textline)
    textline = re.sub('\|LBR\|', '', textline)
    textline = re.sub('\.\.\.+', '...', textline)
    textline = re.sub('!!+', '!!', textline)
    textline = re.sub('\?\?+', '??', textline)
    words = re.compile('[\U00010000-\U0010ffff]|[\w-]+|[^ \w\U00010000-\U0010ffff]+', re.UNICODE).findall(textline.strip())
    words = [w.strip() for w in words if w.strip() != '']
    return(words)

In [ ]:
def random_generator(features, labels, batch_size):
    
    batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
    batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims))
    batch_labels = np.zeros((batch_size, 2))

    while True:
        # print(len(features))
        for i in range(batch_size):
            index = random.choice(len(features), 1)[0]
            batch_features_ft[i], batch_features_lg[i] = process_features(features[index], nb_sequence_length, nb_embedding_dims)
            # print(batch_features[i])
            # print(batch_features[i].shape)
            batch_labels[i] = labels[index]
        yield [batch_features_ft, batch_features_lg], batch_labels

In [18]:
def sequential_generator(filename, batch_size):
    
    file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
    shuffled_indexes = range(1, file_length + 1) # start from 1 because linecache is 1 indexed
#     shuffle(shuffled_indexes)
    
    index_position = 0
    
    batch_features_ft = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
#     batch_features_lg = np.zeros((batch_size, nb_sequence_length, nb_embedding2_dims)) # levy goldberg: not needed
    batch_labels = np.zeros((batch_size, 2)) # 2 because of simple task

    while True:
        # print(len(features))
        
        for i in range(batch_size):
            line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            batch_features_ft[i] = process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            index_position += 1
            if index_position == file_length:
                # shuffle indexes again
                shuffled_indexes = range(1, file_length + 1)
#                 shuffle(shuffled_indexes)
                index_position = 0
                break
        yield [batch_features_ft], batch_labels
        # yield [batch_features_ft], batch_labels

In [6]:
# word_splitter = re.compile("[\w+]|[\W+]", re.UNICODE)
word_vectors_ft = {}
def process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    # print(words)
    features_ft = np.zeros((nb_sequence_length, nb_embedding_dims))
#     features_lg = np.zeros((nb_sequence_length, nb_embedding2_dims))
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        features_ft[idx] = wv
        
#         if w in word2Idx:
#             wv = wordEmbeddings[word2Idx[w]]
#         else:
#             wv = wordEmbeddings[word2Idx["UNKNOWN_TOKEN"]]
#         features_lg[idx] = wv
        
        idx = idx + 1
    return features_ft

In [ ]:
def charcnn_sequential_generator(filename, batch_size):
    
    file_length = sum(1 for line in open(filename, encoding = 'UTF-8'))
    shuffled_indexes = list(range(1, file_length + 1))
    shuffle(shuffled_indexes)
    index_position = 0
    
    # print("INITIIALIZING BATCHHH")

    while True:
        
        # print("PROOODUCING  BATCHHH")
        
        batch_word_embeddings = np.zeros((batch_size, nb_sequence_length, nb_embedding_dims))
        batch_char_embeddings = []
        batch_labels = np.zeros((batch_size, 2)) # 2 for the simple task
    
        for i in range(batch_size):
            line = linecache.getline(filename, shuffled_indexes[index_position])
            data = line.strip().split('\t')
            batch_word_embeddings[i], tmp_char_embeddings = charcnn_process_features(data[0], nb_sequence_length, nb_embedding_dims)
            # print(tmp_char_embeddings.shape)
            batch_char_embeddings.append(tmp_char_embeddings)
            # print(batch_features_ft[i])
            # print(batch_features_ft[i].shape)
            batch_labels[i] = to_categorical(0 if data[1] == 'OTHER' else 1, n_labels)
            index_position += 1
            if index_position == file_length:
                # shuffle indexes again
                shuffled_indexes = range(1, file_length + 1)
                shuffle(shuffled_indexes)
                index_position = 0
                break
        if len(batch_char_embeddings) < batch_size:
            batch_word_embeddings = batch_word_embeddings[:len(batch_char_embeddings)]
            batch_labels = batch_labels[:len(batch_char_embeddings)]
        yield [batch_word_embeddings, np.array(batch_char_embeddings)], batch_labels

word_vectors_ft = {}
def charcnn_process_features(textline, nb_sequence_length, nb_embedding_dims):
    words = twitter_tokenizer(textline)
    # print(words)
    word_embeddings = np.zeros((nb_sequence_length, nb_embedding_dims))
    char_embeddings = np.zeros((nb_sequence_length, 52))
    max_words = min(len(words), nb_sequence_length)
    idx = nb_sequence_length - len(words[:max_words])
    for w in words[:max_words]:
        if w in word_vectors_ft:
            wv = word_vectors_ft[w]
        else:
            wv = ft.get_word_vector(w.lower())
            word_vectors_ft[w] = wv
        word_embeddings[idx] = wv
        
        temp_char = []
        for char in w:
            temp_char.append(char2Idx[char])
        for pos, c in enumerate(temp_char):
            char_embeddings[idx][pos] = c

        idx = idx + 1
    return word_embeddings, char_embeddings

In [7]:
train_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_train.txt', encoding = "UTF-8")]
dev_lines = [line.strip().split("\t") for line in open('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', encoding = "UTF-8")]

In [8]:
train_sentences = [x[0] for x in train_lines]
train_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in train_lines])
# train_labels = [0 if x[1] == "OTHER" else 1 for x in train_lines]

dev_sentences = [x[0] for x in dev_lines]
dev_labels = to_categorical([0 if x[1] == "OTHER" else 1 for x in dev_lines])
# dev_labels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]

In [9]:
n_labels = 2

In [10]:
characters={}
for line in train_sentences:
    for char in line:
        characters[char] = True
for line in dev_sentences:
    for char in line:
        characters[char] = True
char2Idx={}
for char in characters:
    char2Idx[char] = len(char2Idx)

In [ ]:
word2Idx = {}
wordEmbeddings = []

# fEmbeddings = open("../embeddings/model_levy_goldberg_extended.embeddings", encoding="UTF-8")
fEmbeddings = open("../embeddings/embed_tweets_de_300M_52D", encoding="UTF-8")
for line in fEmbeddings:
    split = line.strip().split(" ")
    word = split[0]
    if len(split)-1 != 52:
        continue
    
    if len(word2Idx) == 0: #Add padding+unknown
        word2Idx["PADDING_TOKEN"] = len(word2Idx)
        vector = np.zeros(len(split)-1) #Zero vector vor 'PADDING' word
        wordEmbeddings.append(vector)
        
        word2Idx["UNKNOWN_TOKEN"] = len(word2Idx)
        vector = np.random.uniform(-0.25, 0.25, len(split)-1)
        wordEmbeddings.append(vector)
    
    vector = np.array([float(num) for num in split[1:]])
    wordEmbeddings.append(vector)
    word2Idx[split[0]] = len(word2Idx)

wordEmbeddings = np.array(wordEmbeddings)

In [ ]:
nb_embedding2_dims = wordEmbeddings[1].shape[0]
print(nb_embedding2_dims)
print('für' in word2Idx)
print(wordEmbeddings[word2Idx['für']])
print('ute' in word2Idx)
print(wordEmbeddings[word2Idx['ute']])


In [ ]:
model = Sequential([
    LSTM(64, recurrent_dropout = 0.5, dropout = 0.5, activation = 'relu', input_shape=(nb_sequence_length, nb_embedding_dims)),
    Dense(32, activation = 'relu'),
    Dropout(0.2),
    Dense(2, activation = 'softmax')
])
model.compile(loss='sparse_categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])

In [ ]:
# model = Sequential([
#     Conv1D(128, kernel_size = 3, padding = 'valid', input_shape=(nb_sequence_length, nb_embedding_dims), activation = 'relu'),
#     MaxPooling1D(5),
#     Flatten(),
#     Dense(64, activation = 'relu'),
#     Dropout(0.2),
#     Dense(2, activation = 'softmax')
# ])
# model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])

In [19]:
filter_sizes = (3, 4, 5)
model_input_ft = Input(shape = (nb_sequence_length, nb_embedding_dims))
# model_layers = Dropout(0.1)(model_input)
conv_blocks = []
for sz in filter_sizes:
    conv = Conv1D(
        filters = 200,
        kernel_size = sz,
        padding = 'valid',
        strides = 1
    )(model_input_ft)
    conv = LeakyReLU()(conv)
    conv = GlobalMaxPooling1D()(conv)
    conv = Dropout(0.8)(conv)
    # conv = Flatten()(conv)
    conv_blocks.append(conv)
# model_input_lg = Input(shape = (nb_sequence_length, nb_embedding2_dims))
# model_input_ft = Input(shape = (nb_sequence_length, nb_embedding_dims))
# Variation 1: 
lstm_block = LSTM(100)(model_input_ft) # , dropout = 0.5, recurrent_dropout = 0.5
lstm_block = LeakyReLU()(lstm_block)
# Variation 2: 
# lstm_block = Bidirectional(LSTM(100))(model_input_lg)
# lstm_block = LeakyReLU()(lstm_block)
# Variation 3: 
# lstm_block = attention_3d_block(model_input)
# lstm_block = LSTM(100)(lstm_block)
# lstm_block = LeakyReLU()(lstm_block)
# Variation 4:
# lstm_block = LSTM(100, return_sequences = True)(model_input_ft)
# lstm_block = LeakyReLU()(lstm_block)
# lstm_block = AttentionDecoder(100, nb_sequence_length)(lstm_block)
# lstm_block = Flatten()(lstm_block)
model_concatenated = concatenate([lstm_block, conv_blocks[0], conv_blocks[1], conv_blocks[2]])
# model_concatenated = Dropout(0.8)(model_concatenated)
model_concatenated = Dense(64)(model_concatenated)
model_concatenated = LeakyReLU()(model_concatenated)
model_output = Dense(n_labels, activation = "softmax")(model_concatenated)
model = Model([model_input_ft], model_output)
model.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            (None, 75, 300)      0                                            
__________________________________________________________________________________________________
conv1d_7 (Conv1D)               (None, 73, 200)      180200      input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_8 (Conv1D)               (None, 72, 200)      240200      input_3[0][0]                    
__________________________________________________________________________________________________
conv1d_9 (Conv1D)               (None, 71, 200)      300200      input_3[0][0]                    
__________________________________________________________________________________________________
leaky_re_l

In [ ]:
# word embeddings
words_input = Input(shape=(None,300), dtype='float32',name='words_input')
# character embeddings
character_input=Input(shape=(None,52,),name='char_input')
embed_char_out=TimeDistributed(Embedding(len(char2Idx),30,embeddings_initializer=RandomUniform(minval=-0.5, maxval=0.5)), name='char_embedding')(character_input)
dropout= Dropout(0.5, name='dropout1')(embed_char_out)
conv1d_out= TimeDistributed(Conv1D(kernel_size=3, filters=30, padding='same',activation='tanh', strides=1, name='conv'))(dropout)
maxpool_out=TimeDistributed(MaxPooling1D(52), name='maxpool')(conv1d_out)
char = TimeDistributed(Flatten())(maxpool_out)
char = Dropout(0.5)(char)
# concatenation
output = concatenate([words_input, char])
output = Bidirectional(LSTM(200, return_sequences=False, dropout=0.50, recurrent_dropout=0.5))(output)
output = Dense(n_labels, activation='softmax')(output)
model_charcnn = Model(inputs=[words_input, character_input], outputs=[output])
model_charcnn.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
model_charcnn.summary()

In [12]:
samples_per_epoch = len(train_sentences)
epochs = 50
batch_size = 32
steps_per_epoch = math.ceil(samples_per_epoch / batch_size)
checkpoint = ModelCheckpoint('best_classification_model.h5', 
                             monitor='val_acc', 
#                              verbose = 1, 
                             save_best_only = True, 
                             save_weights_only = True)

In [ ]:
for a, b in charcnn_sequential_generator('../OffLang/sample_train.txt', 32):
    # featuretype {0,1}, batch_size, words, char index vector
    print(a[1].shape)
    print(a[1][0].shape)
    print(a[1][0][0].shape)
    print(a[1][0][0])
    break

In [20]:
dev_batch_size = len(dev_sentences)
model.fit_generator(
    sequential_generator('/home/gwiedemann/notebooks//OffLang/sample_train.txt', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = sequential_generator('/home/gwiedemann/notebooks/OffLang/sample_dev.txt', dev_batch_size),
    validation_steps = math.ceil(len(dev_sentences) / dev_batch_size),
    callbacks = [checkpoint]
)
model.reset_states()

Epoch 1/50
131/132 [============================>.] - ETA: 0s - loss: 0.8352 - acc: 0.6458
Epoch 00001: val_acc did not improve
132/132 [==============================] - 18s 139ms/step - loss: 0.8397 - acc: 0.6456 - val_loss: 0.7480 - val_acc: 0.7153
Epoch 2/50
131/132 [============================>.] - ETA: 0s - loss: 0.6414 - acc: 0.7366
Epoch 00002: val_acc did not improve
132/132 [==============================] - 17s 127ms/step - loss: 0.6410 - acc: 0.7360 - val_loss: 0.6195 - val_acc: 0.6646
Epoch 3/50
131/132 [============================>.] - ETA: 0s - loss: 0.5101 - acc: 0.7736
Epoch 00003: val_acc improved from 0.74257 to 0.75248, saving model to best_classification_model.h5
132/132 [==============================] - 17s 127ms/step - loss: 0.5104 - acc: 0.7730 - val_loss: 0.4937 - val_acc: 0.7525
Epoch 4/50
131/132 [============================>.] - ETA: 0s - loss: 0.4623 - acc: 0.8115
Epoch 00004: val_acc improved from 0.75248 to 0.76114, saving model to best_classification

Epoch 32/50
131/132 [============================>.] - ETA: 0s - loss: 0.1434 - acc: 0.9847
Epoch 00032: val_acc did not improve
132/132 [==============================] - 20s 152ms/step - loss: 0.1423 - acc: 0.9848 - val_loss: 1.2835 - val_acc: 0.7822
Epoch 33/50
131/132 [============================>.] - ETA: 0s - loss: 0.1749 - acc: 0.9726
Epoch 00033: val_acc did not improve
132/132 [==============================] - 19s 145ms/step - loss: 0.1752 - acc: 0.9721 - val_loss: 0.6543 - val_acc: 0.7611
Epoch 34/50
131/132 [============================>.] - ETA: 0s - loss: 0.1654 - acc: 0.9750
Epoch 00034: val_acc did not improve
132/132 [==============================] - 20s 148ms/step - loss: 0.1651 - acc: 0.9749 - val_loss: 1.3938 - val_acc: 0.7649
Epoch 35/50
131/132 [============================>.] - ETA: 0s - loss: 0.0786 - acc: 0.9893
Epoch 00035: val_acc did not improve
132/132 [==============================] - 20s 152ms/step - loss: 0.0781 - acc: 0.9893 - val_loss: 1.3531 - val_

In [21]:
model.load_weights('best_classification_model.h5')

In [ ]:
dev_batch_size = len(dev_sentences)
model_charcnn.fit_generator(
    charcnn_sequential_generator('../OffLang/sample_train.txt', batch_size), 
    steps_per_epoch=steps_per_epoch, epochs=epochs,
    validation_data = charcnn_sequential_generator('../OffLang/sample_dev.txt', dev_batch_size),
    validation_steps = math.ceil(len(dev_sentences) / dev_batch_size),
    callbacks = [checkpoint]
)
model_charcnn.reset_states()

In [ ]:
model_charcnn.load_weights('best_classification_model.h5')
model_charcnn.compile(loss='categorical_crossentropy', optimizer='nadam', metrics = ['accuracy'])
testset_features_e1 = np.zeros((len(dev_sentences), nb_sequence_length, nb_embedding_dims))
testset_features_e2 = np.zeros((len(dev_sentences), nb_sequence_length, 52))   
for i in range(len(dev_sentences)):
    testset_features_e1[i], testset_features_e2[i] = charcnn_process_features(dev_sentences[i], nb_sequence_length, nb_embedding_dims)
results = model_charcnn.predict([testset_features_e1, testset_features_e2])

In [ ]:
idx2Label = {0 : "OTHER", 1 : "OFFENSIVE"}
predLabels = results.argmax(axis=-1)
devLabels = [0 if x[1] == "OTHER" else 1 for x in dev_lines]
# print(idx2Label)
# print(predLabels)
# print(devLabels)
f1 = f1_score(devLabels, predLabels, average='binary', pos_label=1)
r = recall_score(devLabels, predLabels, average='binary', pos_label=1)
p = precision_score(devLabels, predLabels, average='binary', pos_label=1)
a = accuracy_score(devLabels, predLabels)
print("Test-Data: Prec: %.3f, Rec: %.3f, F1: %.3f, Acc: %.3f" % (p, r, f1, a))

In [ ]:
# FP / NP